<a href="https://colab.research.google.com/github/somkiatth/git/blob/master/ku-MLlib_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 45 kB/s 
     |████████████████████████████████| 199 kB 51.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=18c7265f6015667bf536c02a972b98a08ecc240911e7926831e400e11f4e7502
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('ML-assignment').getOrCreate()

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/Datasets/Telco-Customer-Churn.csv',
                    header=True, inferSchema=True)

In [ ]:
df.printSchema()

root
 |-- customerID: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: double (nullable = true)
 |-- TotalCharges: string (nullable = true)
 |-- Churn: string (nullable = true)



In [ ]:
df.count()

7043

In [ ]:
df.sample(5/7000).show()

+----------+------+-------------+-------+----------+------+------------+-------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|tenure|PhoneService|MultipleLines|InternetService|     OnlineSecurity|       OnlineBackup|   DeviceProtection|        TechSupport|        StreamingTV|    StreamingMovies|      Contract|PaperlessBilling|       PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------+----------------+--------------------+--------------+------------+-----+
|2545-LXYVJ|  Male|            0|    Yes|        No|    72|         Ye

# Train-test split

In [ ]:
df_train, df_test = df.randomSplit([0.8, 0.2])

# select cols

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler,\
                               StringIndexer, OneHotEncoder

In [ ]:
num_cols = ['tenure', 'MonthlyCharges']
cat_cols = ['gender', 'InternetService', 'Contract']

In [ ]:
va1_out = 'num_features'
std_out = 'scaled_features'

# set transformers

In [ ]:
va1 = VectorAssembler(inputCols=num_cols, outputCol=va1_out)

In [ ]:
std = StandardScaler(inputCol=va1_out, outputCol=std_out)

In [ ]:
si_out = [c+'_index' for c in cat_cols]
si_out

['gender_index', 'InternetService_index', 'Contract_index']

In [ ]:
si = StringIndexer(inputCols=cat_cols, outputCols=si_out)

In [ ]:
ohe_in = si_out[1:]
ohe_in

['InternetService_index', 'Contract_index']

In [ ]:
ohe_out = [c+'_ohe' for c in ohe_in]
ohe_out

['InternetService_index_ohe', 'Contract_index_ohe']

In [ ]:
ohe = OneHotEncoder(inputCols=ohe_in, outputCols=ohe_out)

In [ ]:
va2_in = [std_out] + si_out[:1] + ohe_out
va2_in

['scaled_features',
 'gender_index',
 'InternetService_index_ohe',
 'Contract_index_ohe']

In [ ]:
va2 = VectorAssembler(inputCols=va2_in, outputCol='features')

In [ ]:
si_y = StringIndexer(inputCol='Churn', outputCol='label')

# LogisticRegression and Pipeline

In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(maxIter=1000)

In [ ]:
from pyspark.ml.pipeline import Pipeline

In [ ]:
stages = [va1, std, si, ohe, va2, si_y, lr]

In [ ]:
lr_pipeline = Pipeline(stages=stages)

# Fit and evaluate

In [ ]:
lr_model = lr_pipeline.fit(df_train)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [ ]:
df_predict_train = lr_model.transform(df_train)

In [ ]:
acc_eval.evaluate(df_predict_train)

0.7850284495021337